In [1]:
# Importing all needed functions
import os
import io
from contextlib import redirect_stdout
from astropy.io import fits
import glob
from Calculating_det_angles import estimate_source_angles_detectors  # Importing ma'am's function
from Tools import tools
import threading
from queue import Queue
import pandas as pd
import numpy as np
from keras.models import load_model
from pathlib import Path
import keras
import traceback

# Load the model
model = load_model(r"C:\Users\arpan\OneDrive\Documents\GRB\models\m05-04-1\m05-04-1.keras")

# getting the candidates
df = pd.read_csv('UNCERT.csv')

# Assuming 'df' is your DataFrame and 'column_name' is the name of the column
event_names = df['event_name'].values.tolist()
event_types = df['event_type'].values.tolist()
reliability = df['reliability'].values.tolist()

dir_path = tools.json_path(r'data_path.json')

# list of bin sizes
bin_list = [0.001,0.005,0.01, 0.1, 0.5, 1, 5]

# time interval around trigger
ti = [-10,50]
t = ti[1] - ti[0]

true_labels = {'GRB': [1,0,0,0],'TGF': [0,1,0,0],'SGR': [0,0,1,0],'SFLARE': [0,0,0,1]}

def get_flags(y_true , y_pred):    
    y_pred = np.array(y_pred)
    max_index, max_value = tools.list_max(y_pred)

    if np.all(y_pred < 0.3):
        return True

    if y_true.index(max(y_true)) == max_index and max_value >= 0.7:
        return False
    elif max_value > 0.9:
        return True
    else:
        return False   

# Function to process a single event
def process_event(event_name, event_type,thread_num):
    try:
        # getting the true label of the event
        if event_type in true_labels.keys():
            y_true = true_labels[event_type]
        elif event_type == 'UNCERT':
            y_true = [0,0,0,0]
        else:
            # returning if the event is not GRB, TGF, SGR, or SFLARE
            return 
        
        # creating a temporary folder to store the files
        temp_thread = r'temp'+ str(thread_num)
        temp_path = os.path.join(dir_path, temp_thread)
        tools.create_folder(temp_path)
        event = event_name
        year = '20' + event[2:4] + "/"

        # URL of the file you want to download
        url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_trigdat_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'
        tools.run_wget_download(url, temp_path)

        # Finding Trigdat file
        trig_string = "_trigdat_"
        trig_pattern = os.path.join(temp_path, 'current', f"*{trig_string}*")
        trigdat_file = glob.glob(trig_pattern)

        # Get the spacecraft pointing from here
        event_filename = trigdat_file[0]

        # Getting the RA and DEC
        with fits.open(event_filename, memmap=True) as pha_list:
            ra_obj, dec_obj = (pha_list[0].header['RA_OBJ']), (pha_list[0].header['DEC_OBJ'])

        brightest_nai, bright_nais, brightest_bgo = estimate_source_angles_detectors.angle_to_grb(ra_obj, dec_obj,event_filename)

        for download_nai in  bright_nais:
            # URL of the tte file to download
            url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_tte_' + download_nai + '_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'

            # Construct the wget command
            tools.run_wget_download(url, os.path.join(temp_path, event_type + '_' + event))

        # Use the glob module to search for TTE files in the directory
        target_string = "_tte_"
        file_pattern = str(Path(temp_path) / Path(event_type + '_' + event) / Path('current') / f"*{target_string}*")
        NaI_detectors = glob.glob(file_pattern)

        print(NaI_detectors)

        for NaI_detector in NaI_detectors:
            if brightest_nai in NaI_detector:
                print(NaI_detector)
                # fetching data
                with fits.open(NaI_detector, memmap=True) as hdul:
                    # energy_channel_data = hdul[1].data
                    all_count_data = hdul[2].data
                    trigtime = hdul[2].header['TRIGTIME']

        counts = all_count_data['TIME'].astype(float) - trigtime

        X = {}
        data = []
        for i in bin_list:
            # Define the range and number of bins
            range_min = ti[0]
            range_max = ti[-1]
                
            bin_size = i

            # Create bin edges
            bin_edges = np.arange(range_min, range_max, bin_size)

            # Create the histogram using numpy.histogram
            hist, edges = np.histogram(counts, bins=bin_edges)

            data = data + list(hist)
        
        X = {str(key): [] for key in bin_list}

        f = 0
        for k in X.keys():
            X[k].append(data[f:f+int(t/float(k))])
            f = f + int(t/float(k))
        x={}

        for key in X.keys():
            arr = np.array(X[key][0])
            arr = arr.reshape(1,arr.shape[0],1)
            x[key] = arr

        y_pred = model.predict(x)
        y_pred = y_pred[0]
        
        # finding if the y_pred is interesting
        print(get_flags(y_true, y_pred))
        if get_flags(y_true, y_pred):
            interesting_events.append(event_name)
            print('interesting event')
            print(event_name)
            print(y_true)
            print(y_pred)
            print('-----------------------------')
        elif y_pred[0] > 0.9:
            GRB_candidates.append(event_name)
            print('GRB candidate')
            print(event_name)
            print(y_true)
            print(y_pred)
            print('-----------------------------')


        # saving the outputs
        output_event_names.append(event_name)
        output_event_types.append(event_type)
        output_event_preds.append(y_pred)
        output_event_trues.append(y_true)

        # Print the size of the queue
        print(event_queue.qsize()) 

    except Exception as e:
        traceback.print_exc()
        print(f"Error processing event {event_name}: {e}")

# Create a queue to store the events
event_queue = Queue()

# Function to run the worker threads
def worker(thread_num):
    print(f"Worker thread {thread_num} started.")
    while True:
        # Get an event from the queue
        event_name, event_type = event_queue.get()

        # Process the event
        process_event(event_name, event_type,thread_num)

        # Signal that the task is completed
        event_queue.task_done()
        print(f"Worker thread {thread_num} processed event {event_name}.")


# intializing the outputs
output_event_names = []
output_event_types = []
output_event_preds = []
output_event_trues = []
interesting_events = []
GRB_candidates = []

# Create and start the worker threads
num_threads = 8
threads = []
for i in range(num_threads):
    worker_thread = threading.Thread(target=worker, args=(i,), daemon=True)
    worker_thread.start()
    threads.append(worker_thread)

# Add events to the queue
for event_name, event_type in zip(event_names[30:], event_types[30:]):
    event_queue.put((event_name, event_type))

# Wait for all events to be processed
event_queue.join()
print("All events have been processed.")

print('done')

c:\Users\arpan\miniconda3\lib\site-packages\keras\src\saving\saving_lib.py:394: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 48 variables whereas the saved optimizer has 94 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Worker thread 0 started.
Worker thread 1 started.
Worker thread 2 started.
Worker thread 3 started.
Worker thread 4 started.
Worker thread 5 started.
Worker thread 6 started.
Worker thread 7 started.
C:\Users\arpan\OneDrive\Documents\GRB\data\temp0
Worker thread 1 processed event bn120614887.
C:\Users\arpan\OneDrive\Documents\GRB\data\temp1
C:\Users\arpan\OneDrive\Documents\GRB\data\temp2
C:\Users\arpan\OneDrive\Documents\GRB\data\temp4
C:\Users\arpan\OneDrive\Documents\GRB\data\temp5
C:\Users\arpan\OneDrive\Documents\GRB\data\temp6
C:\Users\arpan\OneDrive\Documents\GRB\data\temp7
C:\Users\arpan\OneDrive\Documents\GRB\data\temp3
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp1' created successfully.
download start
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp6' created successfully.
download start
Fo

Traceback (most recent call last):
  File "C:\Users\arpan\AppData\Local\Temp\ipykernel_27300\3851385108.py", line 114, in process_event
    counts = all_count_data['TIME'].astype(float) - trigtime
UnboundLocalError: local variable 'all_count_data' referenced before assignment


Downloaded bn110712284_trigdat_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp2
download start
Downloaded bn211002039_trigdat_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp1
download start
Downloaded bn110206749_tte_n2_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp3\UNCERT_bn110206749
download start
Downloaded bn150908768_tte_n2_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp0\UNCERT_bn150908768
['C:\\Users\\arpan\\OneDrive\\Documents\\GRB\\data\\temp0\\UNCERT_bn150908768\\current\\glg_tte_n0_bn150908768_v00.fit', 'C:\\Users\\arpan\\OneDrive\\Documents\\GRB\\data\\temp0\\UNCERT_bn150908768\\current\\glg_tte_n1_bn150908768_v00.fit', 'C:\\Users\\arpan\\OneDrive\\Documents\\GRB\\data\\temp0\\UNCERT_bn150908768\\current\\glg_tte_n2_bn150908768_v00.fit']
C:\Users\arpan\OneDrive\Documents\GRB\data\temp0\UNCERT_bn150908768\current\glg_tte_n0_bn150908768_v00.fit
C:\Users\arpan\OneDrive\Documents\GRB\data\temp0\UNCERT_bn150908768\current\glg_tte_n1_bn150908768_v00.fit
C:\Users

Traceback (most recent call last):
  File "C:\Users\arpan\AppData\Local\Temp\ipykernel_27300\3851385108.py", line 114, in process_event
    counts = all_count_data['TIME'].astype(float) - trigtime
UnboundLocalError: local variable 'all_count_data' referenced before assignment


...

Error downloading the file: Command 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_tte_na_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2022/bn220603728/current/ -P C:\Users\arpan\OneDrive\Documents\GRB\data\temp3\UNCERT_bn220603728' returned non-zero exit status 4.
download start
Error downloading the file: Command 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_tte_n9_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2010/bn100101783/current/ -P C:\Users\arpan\OneDrive\Documents\GRB\data\temp4\UNCERT_bn100101783' returned non-zero exit status 4.
download start
Error downloading the file: Command 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_tte_n5_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2022/bn220603728/current/ -P C:\Users\arpan\OneDrive\Docu

Traceback (most recent call last):
  File "C:\Users\arpan\AppData\Local\Temp\ipykernel_27300\3851385108.py", line 114, in process_event
    counts = all_count_data['TIME'].astype(float) - trigtime
UnboundLocalError: local variable 'all_count_data' referenced before assignment
Traceback (most recent call last):
  File "C:\Users\arpan\AppData\Local\Temp\ipykernel_27300\3851385108.py", line 114, in process_event
    counts = all_count_data['TIME'].astype(float) - trigtime
UnboundLocalError: local variable 'all_count_data' referenced before assignment


Error downloading the file: Command 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_tte_n2_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2012/bn120727408/current/ -P C:\Users\arpan\OneDrive\Documents\GRB\data\temp3\UNCERT_bn120727408' returned non-zero exit status 4.
download start
Error downloading the file: Command 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_tte_na_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2012/bn120727408/current/ -P C:\Users\arpan\OneDrive\Documents\GRB\data\temp3\UNCERT_bn120727408' returned non-zero exit status 4.
download start
Error downloading the file: Command 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_tte_n5_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2012/bn120727408/current/ -P C:\Users\arpan\OneDrive\Docu

Traceback (most recent call last):
  File "C:\Users\arpan\AppData\Local\Temp\ipykernel_27300\3851385108.py", line 114, in process_event
    counts = all_count_data['TIME'].astype(float) - trigtime
UnboundLocalError: local variable 'all_count_data' referenced before assignment


Downloaded bn220609520_tte_nb_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp5\UNCERT_bn220609520
download start
Downloaded bn160228158_tte_n1_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp6\UNCERT_bn160228158
download start
Downloaded bn210525596_tte_n2_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp0\UNCERT_bn210525596
['C:\\Users\\arpan\\OneDrive\\Documents\\GRB\\data\\temp0\\UNCERT_bn210525596\\current\\glg_tte_n0_bn210525596_v00.fit', 'C:\\Users\\arpan\\OneDrive\\Documents\\GRB\\data\\temp0\\UNCERT_bn210525596\\current\\glg_tte_n1_bn210525596_v00.fit', 'C:\\Users\\arpan\\OneDrive\\Documents\\GRB\\data\\temp0\\UNCERT_bn210525596\\current\\glg_tte_n2_bn210525596_v00.fit']
C:\Users\arpan\OneDrive\Documents\GRB\data\temp0\UNCERT_bn210525596\current\glg_tte_n1_bn210525596_v00.fit
Downloaded bn121210642_tte_n3_ to C:\Users\arpan\OneDrive\Documents\GRB\data\temp2\UNCERT_bn121210642
['C:\\Users\\arpan\\OneDrive\\Documents\\GRB\\data\\temp2\\UNCERT_bn121210642\\current\\glg_tt

In [2]:
print(GRB_candidates) # ['bn110102432', 'bn150101576', 'bn100729415', 'bn150711543', 'bn150707234', 'bn150914103', 'bn170815526'] [0.9720139  0.0021333  0.00373149 0.02212139]  [0.94786453 0.0044799  0.00902045 0.03863509]

['bn110102432', 'bn150101576', 'bn100729415', 'bn150711543', 'bn150707234', 'bn150914103', 'bn170815526']


In [3]:
print(interesting_events) # ['bn100530292', 'bn211217424', 'bn210403738', 'bn120813978', 'bn120502856', 'bn240320700', 'bn210425089', 'bn231225841', 'bn171003256', 'bn150316434', 'bn230623728', 'bn171202587', 'bn140330899', 'bn220906529', 'bn190326747', 'bn090523133', 'bn160116302', 'bn120312392', 'bn230912087', 'bn190919899', 'bn120725321', 'bn161111083', 'bn131117611', 'bn140504334', 'bn181010392', 'bn240130149', 'bn210818917', 'bn131101050', 'bn221104472', 'bn170712214', 'bn230613572', 'bn151015141', 'bn221214088', 'bn180827314', 'bn090209200', 'bn140513553', 'bn190620579', 'bn240308556', 'bn220619154', 'bn221011605', 'bn220808197', 'bn230217875', 'bn170603580', 'bn151002067', 'bn090818453', 'bn080922709', 'bn221023281', 'bn210504483', 'bn180904763', 'bn220222492', 'bn240206567', 'bn181209226', 'bn210910456', 'bn140828673', 'bn121015799', 'bn160523504', 'bn090602780', 'bn140619498', 'bn120204771', 'bn130424231', 'bn100322678', 'bn180406562']

['bn100530292', 'bn211217424', 'bn210403738', 'bn120813978', 'bn120502856', 'bn240320700', 'bn210425089', 'bn231225841', 'bn171003256', 'bn150316434', 'bn230623728', 'bn171202587', 'bn140330899', 'bn220906529', 'bn190326747', 'bn090523133', 'bn160116302', 'bn120312392', 'bn230912087', 'bn190919899', 'bn120725321', 'bn161111083', 'bn131117611', 'bn140504334', 'bn181010392', 'bn240130149', 'bn210818917', 'bn131101050', 'bn221104472', 'bn170712214', 'bn230613572', 'bn151015141', 'bn221214088', 'bn180827314', 'bn090209200', 'bn140513553', 'bn190620579', 'bn240308556', 'bn220619154', 'bn221011605', 'bn220808197', 'bn230217875', 'bn170603580', 'bn151002067', 'bn090818453', 'bn080922709', 'bn221023281', 'bn210504483', 'bn180904763', 'bn220222492', 'bn240206567', 'bn181209226', 'bn210910456', 'bn140828673', 'bn121015799', 'bn160523504', 'bn090602780', 'bn140619498', 'bn120204771', 'bn130424231', 'bn100322678', 'bn180406562']


In [2]:
GRB_candidates = ['bn110102432', 'bn150101576', 'bn100729415', 'bn150711543', 'bn150707234', 'bn150914103', 'bn170815526']
interesting_events = ['bn100530292', 'bn211217424', 'bn210403738', 'bn120813978', 'bn120502856', 'bn240320700', 'bn210425089', 'bn231225841', 'bn171003256', 'bn150316434', 'bn230623728', 'bn171202587', 'bn140330899', 'bn220906529', 'bn190326747', 'bn090523133', 'bn160116302', 'bn120312392', 'bn230912087', 'bn190919899', 'bn120725321', 'bn161111083', 'bn131117611', 'bn140504334', 'bn181010392', 'bn240130149', 'bn210818917', 'bn131101050', 'bn221104472', 'bn170712214', 'bn230613572', 'bn151015141', 'bn221214088', 'bn180827314', 'bn090209200', 'bn140513553', 'bn190620579', 'bn240308556', 'bn220619154', 'bn221011605', 'bn220808197', 'bn230217875', 'bn170603580', 'bn151002067', 'bn090818453', 'bn080922709', 'bn221023281', 'bn210504483', 'bn180904763', 'bn220222492', 'bn240206567', 'bn181209226', 'bn210910456', 'bn140828673', 'bn121015799', 'bn160523504', 'bn090602780', 'bn140619498', 'bn120204771', 'bn130424231', 'bn100322678', 'bn180406562']

for i in GRB_candidates:
    print(True if i in interesting_events else False)
    if i in interesting_events:
        print(i)

False
False
False
False
False
False
False


In [5]:
for i in interesting_events:
    n = output_event_names.index(i)
    print(output_event_names[n], output_event_preds[n])

bn100530292 [0.03952479 0.90560275 0.00948716 0.04538533]
bn211217424 [0.01523872 0.9179279  0.02230606 0.04452734]
bn210403738 [6.1018136e-04 9.7675031e-01 2.2551634e-03 2.0384362e-02]
bn120813978 [0.02034596 0.9183629  0.00667826 0.05461283]
bn120502856 [6.6444784e-04 9.7297931e-01 4.9012098e-03 2.1455064e-02]
bn240320700 [0.01578333 0.92596644 0.00658581 0.05166436]
bn210425089 [0.00861925 0.9250781  0.03550718 0.03079555]
bn231225841 [0.01375715 0.9014527  0.03476392 0.05002626]
bn171003256 [0.03218342 0.91693324 0.01578147 0.03510191]
bn150316434 [0.01371853 0.934455   0.00987855 0.04194791]
bn230623728 [0.02237054 0.93424153 0.00802462 0.03536334]
bn171202587 [0.01624764 0.94131577 0.00630885 0.03612777]
bn140330899 [3.7626276e-04 9.8465848e-01 1.1513081e-03 1.3813958e-02]
bn220906529 [0.03225416 0.90775365 0.00657472 0.05341746]
bn190326747 [0.03589498 0.91044104 0.02251572 0.03114822]
bn090523133 [0.00789331 0.94459975 0.01843777 0.02906907]
bn160116302 [0.01139054 0.9548733  0

In [ ]:
# False 514
# bn160114314
# [0, 0, 0, 0]
# [0.8616942  0.02299897 0.00460136 0.11070547]
# -----------------------------


# False
# bn230330654
# [0, 0, 0, 0]
# [0.94876885 0.02230632 0.00306564 0.02585921]
# -----------------------------

In [ ]:
# True
# bn080924766
# [1, 0, 0, 0]
# [0.01024451 0.93319297 0.02897306 0.02758948]
# -----------------------------

# True
# bn181212693
# [1, 0, 0, 0]
# [0.00209014 0.97003317 0.01413007 0.0137467 ]
# -----------------------------

# True
# bn080829790
# [1, 0, 0, 0]
# [0.00920342 0.9551996  0.00546318 0.03013378]
# -----------------------------

In [3]:
print(event_names)

['bn090131590', 'bn080815917', 'bn090328545', 'bn080719529', 'bn090122451', 'bn090206897', 'bn090208340', 'bn080924766', 'bn080912360', 'bn090326625', 'bn130525834', 'bn200604778', 'bn090210310', 'bn090122694', 'bn081225257', 'bn090123617', 'bn090122832', 'bn080928628', 'bn090202394', 'bn090122584', 'bn090127632', 'bn090208463', 'bn080805496', 'bn090128024', 'bn090201684', 'bn090126966', 'bn090125277', 'bn090202862', 'bn090210898', 'bn081003385', 'bn081003446', 'bn081003377', 'bn230320150', 'bn090130290', 'bn080823363', 'bn090210415', 'bn090125911', 'bn090129936', 'bn180722120', 'bn140801432', 'bn080916289', 'bn181212693', 'bn090409288', 'bn180804765', 'bn090122898', 'bn080829790', 'bn081003644', 'bn190817953', 'bn081017474']


In [4]:
print(event_names.index('bn181212693'))

41
